In [1]:
import numpy as np 

In [2]:
## Functions for parsing the sum of different functions 

def split_plus(function) :
    terms_list = []
    if '+' in function : 
        index = function.index('+')
        #print('funct1 =',function[:index])
        #print('funct2 =',function[index+1:])
        terms_list.append(function[:index])
        new_funct = split_plus(function[index+1:])
        for element in new_funct : 
            if type(element) == list : 
                terms_list.append(element[0])
            else : 
                terms_list.append(element)
    else : 
        terms_list.append(function)
    
    return terms_list 

def split_minus(terms_list) : 
    for i, term in enumerate(terms_list) : 
        if '-' in term : 
            index = term.index('-')
            terms_list[i] = term[:index]
            terms_list.insert(i+1, term[index+1:])
            new_term = split_minus(terms_list)
        
    return terms_list

def split_pm(function) : 
    sign = [char for char in function if char == '+' or char == '-']
    sign.insert(0, '+')
    funct_list = function.split('+') 
    for i,term in enumerate(funct_list):
        if '-' in term : 
            funct_list[i] = term.split('-')[0] 
            funct_list.insert(i+1, term.split('-')[1])
    return funct_list, sign


        

In [3]:
## Test for the parser function

function = "x1*x2-sin(x1)+cos(x1)-75"
funct_list, sign = split_pm(function)
print(funct_list, sign)

['x1*x2', 'sin(x1)', 'cos(x1)', '75'] ['+', '-', '+', '-']


In [4]:
## Functions for derivative of some functions 

def der1(x1, x2, der_var) : 
    if der_var == 'x1' :
        dx1 = [x1,1]
        dx2 = [x2,0]
    elif der_var == 'x2' : 
        dx1 = [x1,0]
        dx2 = [x2,1]
    return [dx1, dx2]

def der_sum(dx1, dx2) : 
    d_sum = [ dx1[0] + dx2[0], dx1[1] + dx2[1] ]
    return d_sum

def der_diff(dx1, dx2) : 
    d_diff = [ dx1[0] - dx2[0], dx1[1] - dx2[1] ]
    return d_diff

def der_mult(d_dx, list_var, der_var) : 
    for i, var in enumerate(list_var) : 
        if der_var == var : 
            d_prod = [ d_dx[i][0] * d_dx[i-1][0], d_dx[i][0]*d_dx[i-1][1] + d_dx[i][1]*d_dx[i-1][0] ]
    
    return d_prod

def der_sin(d_dx, list_var, der_var) : 
    for i, var in enumerate(list_var) : 
        if der_var == var : 
            d_sin = [ np.sin(d_dx[i][0]), np.cos(d_dx[i][0])*d_dx[i][1] ]
    
    return d_sin

def der_cos(d_dx, list_var, der_var) : 
    for i, var in enumerate(list_var) : 
        if der_var == var : 
            d_cos = [ np.cos(d_dx[i][0]), -1*np.sin(d_dx[i][0])*d_dx[i][1] ]
    
    return d_cos

In [5]:
## Autodiff function 

def derivate(function, list_var, v, w, der_var) : 
    d_dx = der1(x1=v, x2=w, der_var=der_var)
    der_term = []
    funct_list, sign = split_pm(function)

    for term in funct_list : 
        if '*' in term : 
            factors = term.split('*')
            op = []
            for factor in factors :
                for i,var in enumerate(list_var) : 
                    if factor==var : 
                        op.append(d_dx[i])
            d_prod = der_mult(d_dx=op, list_var=list_var, der_var=der_var) 
            der_term.append(d_prod)

        elif 'sin' in term : 
            angle = term.replace('sin(', '')
            angle = angle.replace(')', '')
            d_sin = der_sin(d_dx=d_dx, list_var=list_var, der_var=der_var )
            der_term.append(d_sin)

        elif 'cos' in term : 
            angle = term.replace('cos(', '')
            angle = angle.replace(')', '')
            d_cos = der_cos(d_dx=d_dx, list_var=list_var, der_var=der_var )
            der_term.append(d_cos)
        
    total_der = 0
    for i, addend in enumerate(der_term) :
        if sign[i] == '+' :
            total_der += addend[1]
        elif sign[i] == '-' : 
            total_der -= addend[1]
    
    return der_term, total_der

In [7]:
function = 'x1*x2-cos(x1)+sin(x1)'
der_term, total_der = derivate(function, list_var=['x1','x2'], v=2, w=3, der_var='x1')
print('Derivatives of the addend : ', der_term)
print('Total derivative = ', total_der)

Derivatives of the addend :  [[6, 3], [-0.4161468365471424, -0.9092974268256817], [0.9092974268256817, -0.4161468365471424]]
Total derivative =  3.4931505902785394
